In [ ]:
import numpy as np
import awkward as ak
import uproot

import matplotlib.pylab as plt

import hepfile

import time

import vector
vector.register_awkward()


In [ ]:
f = uproot.open('../../Downloads/Run2012BC_DoubleMuParked_Muons.root')

In [ ]:
f.keys()

In [ ]:
f['Events'].keys()

In [ ]:
start = time.time()

ak_arrays = {}
ak_arrays['pt'] = f['Events']['Muon_pt'].array()
ak_arrays['eta'] = f['Events']['Muon_eta'].array()
ak_arrays['phi'] = f['Events']['Muon_phi'].array()
ak_arrays['mass'] = f['Events']['Muon_mass'].array()
ak_arrays['charge'] = f['Events']['Muon_charge'].array()

print(f"Time to extract data from ROOT file: {time.time()-start} seconds")


start = time.time()

data = hepfile.initialize()

hepfile.awkward_tools.pack_multiple_awkward_arrays(data, ak_arrays, group_name='Muon')

print(f"Time to write data to the data dict: {time.time()-start} seconds")


In [ ]:
#data

In [ ]:
start = time.time()

hepfile.write_to_file('Run2012BC_DoubleMuParked_Muons.h5', data, verbose=False)

print(f"Time to write data to the uncompressed file: {time.time()-start} seconds")


start = time.time()

hepfile.write_to_file('Run2012BC_DoubleMuParked_Muons_COMPRESSED.h5', data, \
                      verbose=False, comp_type='gzip',comp_opts=9)

print(f"Time to write data to the compressed file: {time.time()-start} seconds")


In [ ]:
!ls -ltr Run2012BC_DoubleMuParked_Muons.h5
!ls -ltr Run2012BC_DoubleMuParked_Muons_COMPRESSED.h5

!ls -ltr ../../Downloads/Run2012BC_DoubleMuParked_Muons.root

In [ ]:
start = time.time()

data, _ = hepfile.load('Run2012BC_DoubleMuParked_Muons.h5')

start1 = time.time()

dataAwk = hepfile.awkward_tools.hepfile_to_awkward(data)

print(f"Time to read data into the data dictionary: {start1-start} seconds")
print(f"Time to convert data dictionary to awk: {time.time()-start1} seconds")


In [ ]:
mask = ak.num(dataAwk.Muon.pt)==4

print(mask[0:10])

print(len(mask))


In [ ]:
mask_muons = (dataAwk.Muon.pt>5) & (np.abs(dataAwk.Muon.eta)<2.4)

print(mask_muons[0:10])

print(len(mask_muons))

In [ ]:
#print(len(dataAwk[mask].Muon))

start = time.time()
muons = dataAwk.Muon[mask_muons]
muons = muons[mask]
print(f"Time to do something: {time.time()-start}")


start = time.time()
muons = dataAwk[mask].Muon[mask_muons[mask]]
print(f"Time to do something: {time.time()-start}")


In [ ]:
start = time.time()
muons = ak.zip({
    "pt": muons["pt"],
    "phi": muons["phi"],
    "eta": muons["eta"],
    "mass": muons["mass"],
    "charge": muons["charge"],
}, with_name="Momentum4D")
print(f"Time to do something: {time.time()-start}")

print(type(muons))

start = time.time()
quads = ak.combinations(muons, 4)
print(f"Time to do something: {time.time()-start}")

print(type(quads))


start = time.time()
mu1, mu2, mu3, mu4 = ak.unzip(quads)
print(f"Time to do something: {time.time()-start}")

start = time.time()
p4 = mu1 + mu2 + mu3 + mu4
print(f"Time to do something: {time.time()-start}")



In [ ]:
#doubles = ak.combinations(quads,2)
#zmu1,zmu2 = ak.unzip(doubles)

#zp4 = zmu1 + zmu2

#zcharge = zmu1.charge + zmu2.charge

In [ ]:
start = time.time()
tot_charge = mu1.charge + mu2.charge + mu3.charge + mu4.charge
print(f"Time to do something: {time.time()-start}")

start = time.time()
charge_mask = tot_charge==0
print(f"Time to do something: {time.time()-start}")

DR_CUT = 0.02
start = time.time()
mask_dr = (mu1.deltaR(mu2)>DR_CUT) & \
          (mu1.deltaR(mu3)>DR_CUT) & \
          (mu1.deltaR(mu4)>DR_CUT) & \
          (mu2.deltaR(mu3)>DR_CUT) & \
          (mu2.deltaR(mu4)>DR_CUT) & \
          (mu3.deltaR(mu4)>DR_CUT)

print(f"Time to do something: {time.time()-start}")

In [ ]:
#mu1.deltaR(mu2)
#mask_dr = (mu1.deltaR(mu1)<0.02) & (mu1.deltaR(mu3)<0.02)
mask_dr

In [ ]:
plt.figure()
plt.hist(ak.flatten(p4[charge_mask & mask_dr].mass), bins=36,range=(70,180));

In [ ]:
plt.figure()
plt.hist(ak.flatten(p4[charge_mask].mass), bins=100,range=(0,200));

In [ ]:
start = time.time()
z12 = mu1 + mu2
z34 = mu3 + mu4

q12 = mu1.charge + mu2.charge
q34 = mu3.charge + mu4.charge
print(f"Time to do something: {time.time()-start}")

start = time.time()
z13 = mu1 + mu3
z24 = mu2 + mu4

q13 = mu1.charge + mu3.charge
q24 = mu2.charge + mu4.charge
print(f"Time to do something: {time.time()-start}")

start = time.time()
z14 = mu1 + mu4
z23 = mu2 + mu3

q14 = mu1.charge + mu4.charge
q23 = mu2.charge + mu3.charge
print(f"Time to do something: {time.time()-start}")

start = time.time()
p4 = mu1 + mu2 + mu3 + mu4
print(f"Time to do something: {time.time()-start}")


print(len(z12), len(z34), len(p4))

In [ ]:
DR_CUT = 0.02
start = time.time()
'''
mask_dr = (mu1.deltaR(mu2)>DR_CUT) & \
          (mu1.deltaR(mu3)>DR_CUT) & \
          (mu1.deltaR(mu4)>DR_CUT) & \
          (mu2.deltaR(mu3)>DR_CUT) & \
          (mu2.deltaR(mu4)>DR_CUT) & \
          (mu3.deltaR(mu4)>DR_CUT)
'''

mask = (q12==0) & (q34==0) & (mu1.deltaR(mu2)>DR_CUT) & (mu3.deltaR(mu4)>DR_CUT)
masses = ak.flatten(z12.mass[mask]).tolist() + ak.flatten(z34.mass[mask]).tolist() 

mask = (q13==0) & (q24==0) & (mu1.deltaR(mu3)>DR_CUT) & (mu2.deltaR(mu4)>DR_CUT)
masses += ak.flatten(z13.mass[mask]).tolist() + ak.flatten(z24.mass[mask]).tolist() 

mask = (q14==0) & (q23==0) & (mu1.deltaR(mu4)>DR_CUT) & (mu2.deltaR(mu3)>DR_CUT)
masses += ak.flatten(z14.mass[mask]).tolist() + ak.flatten(z23.mass[mask]).tolist() 
print(f"Time to do something: {time.time()-start}")


plt.hist(masses,bins=100, range=(0,200));

In [ ]:
LOCUT = 40
HICUT = 120

start = time.time()
mask1 = (q12==0) & (q34==0) & (mu1.deltaR(mu2)>DR_CUT) & (mu3.deltaR(mu4)>DR_CUT) & \
         (z12.mass>LOCUT) & (z12.mass<HICUT) &  (z34.mass>LOCUT) & (z34.mass<HICUT)


mask2 = (q13==0) & (q24==0) & (mu1.deltaR(mu3)>DR_CUT) & (mu2.deltaR(mu4)>DR_CUT) & \
         (z13.mass>LOCUT) & (z13.mass<HICUT) &  (z24.mass>LOCUT) & (z24.mass<HICUT)


mask3 = (q14==0) & (q23==0) & (mu1.deltaR(mu4)>DR_CUT) & (mu2.deltaR(mu3)>DR_CUT) & \
         (z14.mass>LOCUT) & (z14.mass<HICUT) &  (z23.mass>LOCUT) & (z23.mass<HICUT)


print(f"Time to do something: {time.time()-start}")

plt.hist(ak.flatten(p4.mass[mask1 | mask2 | mask3]), bins=36, range=(70,180));


In [ ]:
plt.hist(masses,bins=36, range=(70,180));

In [ ]:
plt.hist(ak.flatten(mu1.pt),bins=100, range=(0,100));

In [ ]:
plt.hist(ak.flatten(mu1.eta),bins=100, range=(-5,5));